# 🎥 Face Movie Generator (MediaPipe version)

Cette application recrée l'effet "Face Movie" de Picasa directement dans Google Colab.

## ✨ Nouveautés :
- Détection de visage via **MediaPipe** (plus fiable que Haar)
- Fallback OpenCV Haar si MediaPipe échoue
- Recadrage centré sur le visage
- Zoom et fondu entre les images
- Ajout optionnel de musique
- Téléchargement facile de la vidéo

💡 **Utilisation** :
1. Exécute toutes les cellules
2. Upload tes images et musique
3. Télécharge ta vidéo générée

In [ ]:
# 🛠️ Installer les bibliothèques nécessaires
!pip install gradio opencv-python moviepy numpy mediapipe --quiet

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
from tempfile import TemporaryDirectory

# Initialisation MediaPipe & Haar
mp_face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face(image, size=(640, 640)):
    # Conversion en RGB (MediaPipe attend du RGB)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    h, w, _ = img_rgb.shape

    # === 1) Détection via MediaPipe ===
    results = mp_face_detection.process(img_rgb)
    if results.detections:
        det = results.detections[0]
        bbox = det.location_data.relative_bounding_box
        x1 = max(int(bbox.xmin * w) - 20, 0)
        y1 = max(int(bbox.ymin * h) - 20, 0)
        x2 = min(int((bbox.xmin + bbox.width) * w) + 20, w)
        y2 = min(int((bbox.ymin + bbox.height) * h) + 20, h)
        face_img = image[y1:y2, x1:x2]
        face_img = cv2.resize(face_img, size)
        return face_img

    # === 2) Fallback Haar si MediaPipe échoue ===
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if len(faces) > 0:
        x, y, w_box, h_box = max(faces, key=lambda f: f[2]*f[3])
        margin = 0.4
        x1 = max(int(x - w_box*margin), 0)
        y1 = max(int(y - h_box*margin), 0)
        x2 = min(int(x + w_box + w_box*margin), image.shape[1])
        y2 = min(int(y + h_box + h_box*margin), image.shape[0])
        face_img = image[y1:y2, x1:x2]
        face_img = cv2.resize(face_img, size)
        return face_img

    return None

def create_face_movie(images, music=None):
    with TemporaryDirectory() as tempdir:
        clips = []
        for img in images:
            face = detect_face(img)
            if face is not None:
                clip = ImageClip(face).set_duration(2).resize(lambda t: 1 + 0.03 * t).crossfadein(1)
                clips.append(clip)
        if not clips:
            return "Aucun visage trouvé", None
        video = concatenate_videoclips(clips, method="compose")
        if music:
            video = video.set_audio(AudioFileClip(music.name).set_duration(video.duration))
        path = os.path.join(tempdir, "face_movie.mp4")
        video.write_videofile(path, fps=24, codec="libx264", audio_codec="aac")
        return "Vidéo prête 🎉", path

# Interface Gradio
iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label="Images", file_types=["image"], file_count="multiple"),
        gr.File(label="Musique (optionnel)", file_types=["audio"])
    ],
    outputs=[
        gr.Text(label="Statut"),
        gr.File(label="Télécharger la vidéo")
    ],
    title="Face Movie (MediaPipe)",
    description="Crée un diaporama vidéo centré sur les visages avec MediaPipe — compatible iPad"
)

iface.launch(share=True)